# Golden Dataset Extraction

This notebook constructs a governance-focused subset of the Civil Comments dataset
designed to surface system-level risk in online moderation systems.

The goal is to sample comments that are most informative for evaluating ambiguity,
bias, and evaluation blind spots rather than maximizing dataset size.



In [ ]:
from google.colab import drive

# Mount the drive at the default mount point
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Define the Dataset Path
After successfully mounting the drive, we specify the exact file path to the dataset.
This ensures reproducibility and clarity for future researchers who may run this notebook.
The dataset in this project is the *Civil Comments TFDS* file located within the folder structure:
`MyDrive/Dat490/Dataset/Civil_Comments_TFDS.csv`.
We will store this path in a variable for convenient reference throughout the notebook.


In [ ]:
# Define the absolute path to the Civil Comments dataset
dataset_path = "/content/drive/MyDrive/Dat490/Dataset/Civil_Comments_TFDS.csv"

# Quick verification: print the path to confirm correctness
print("Dataset path set to:", dataset_path)


Dataset path set to: /content/drive/MyDrive/Dat490/Dataset/Civil_Comments_TFDS.csv


## Import Required Libraries
This step imports the core Python libraries that will be used for data exploration and manipulation.  
Each import is kept separate for clarity. The following packages are standard in data-science workflows:

- **pandas** – for data-frame manipulation and analysis  
- **numpy** – for numerical operations and array handling  
- **matplotlib / seaborn** (to be added later if visualization is required)


In [ ]:
# Importing essential libraries

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

print(" Libraries successfully imported.")


✅ Libraries successfully imported.


## Dataset Source

This analysis uses the Civil Comments dataset under its original license.
Raw data is not included in this repository.

Users must supply their own local file path to the dataset in order to run this notebook.



In [ ]:
# Load the Civil Comments dataset using the pre-defined path
df = pd.read_csv(dataset_path, encoding='utf-8')

# Basic verification of successful loading
print(" Dataset successfully loaded.")
print(f"Shape: {df.shape}")             # Displays the number of rows and columns
print("Preview of column names:", list(df.columns)[:10], "...")


✅ Dataset successfully loaded.
Shape: (1999514, 14)
Preview of column names: ['article_id', 'created_date', 'id', 'identity_attack', 'insult', 'obscene', 'parent_id', 'parent_text', 'publication_id', 'severe_toxicity'] ...


## Inspect Dataset Structure
Before any filtering or transformation, it is important to examine the dataset’s structure to understand
its overall composition. This includes column names, data types, and the presence of missing values.  


In [ ]:
# Display structural information about the dataset
# Provides data types, non-null counts, and memory usage
df.info()

# Display the first few rows to preview text and label structure
df.head(3)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1999514 entries, 0 to 1999513
Data columns (total 14 columns):
 #   Column           Dtype  
---  ------           -----  
 0   article_id       int64  
 1   created_date     object 
 2   id               object 
 3   identity_attack  float64
 4   insult           float64
 5   obscene          float64
 6   parent_id        int64  
 7   parent_text      object 
 8   publication_id   object 
 9   severe_toxicity  float64
 10  sexual_explicit  float64
 11  text             object 
 12  threat           float64
 13  toxicity         float64
dtypes: float64(7), int64(2), object(5)
memory usage: 213.6+ MB


,article_id,created_date,id,identity_attack,insult,obscene,parent_id,parent_text,publication_id,severe_toxicity,sexual_explicit,text,threat,toxicity
0,153145,b'2016-11-29 17:23:57.762283+00',b'634903',0.0,0.0,0.0,0,b'',b'54',0.0,0.0,"b""btw, Globe, your new comment section is lame...",0.0,0.2
1,379147,b'2017-09-19 03:02:05.207449+00',b'5977874',0.0,0.0,0.0,5977787,"b""I get the impression that Boeing is very af...",b'54',0.0,0.0,"b""If at first you don't succeed...try again: h...",0.0,0.0
2,342612,b'2017-06-10 06:32:21.121964+00',b'5390534',0.0,0.7,0.0,5388737,"b""Here's a vote for Comey to be prosecuting fo...",b'55',0.0,0.0,"b""You don't understand what leaking is. By th...",0.0,0.7


## Decode All Byte-String Columns
Some columns in the dataset (such as text fields or identifiers) may have been stored as
byte objects (e.g., `b"example"`) during export. To standardize the dataset for analysis,
we automatically detect and decode every column containing byte strings.
This ensures that all text fields are human-readable and compatible with downstream
processing without altering numeric values or labels.


In [ ]:
# Automatically detect and decode any column that contains byte strings
# This approach preserves non-text columns (e.g., numeric, datetime)
# and only decodes columns with at least one byte or bytearray entry.

for col in df.columns:
    if df[col].apply(lambda x: isinstance(x, (bytes, bytearray))).any():
        df[col] = df[col].apply(lambda x: x.decode('utf-8', errors='ignore') if isinstance(x, (bytes, bytearray)) else x)
        print(f"Decoded column: {col}")

print("✅ Byte-string decoding completed for all applicable columns.")
df.head(3)


✅ Byte-string decoding completed for all applicable columns.


,article_id,created_date,id,identity_attack,insult,obscene,parent_id,parent_text,publication_id,severe_toxicity,sexual_explicit,text,threat,toxicity
0,153145,b'2016-11-29 17:23:57.762283+00',b'634903',0.0,0.0,0.0,0,b'',b'54',0.0,0.0,"b""btw, Globe, your new comment section is lame...",0.0,0.2
1,379147,b'2017-09-19 03:02:05.207449+00',b'5977874',0.0,0.0,0.0,5977787,"b""I get the impression that Boeing is very af...",b'54',0.0,0.0,"b""If at first you don't succeed...try again: h...",0.0,0.0
2,342612,b'2017-06-10 06:32:21.121964+00',b'5390534',0.0,0.7,0.0,5388737,"b""Here's a vote for Comey to be prosecuting fo...",b'55',0.0,0.0,"b""You don't understand what leaking is. By th...",0.0,0.7


## Clean Residual Byte-like Strings
Certain columns may still contain *string literals* that
look like bytes (e.g., `"b'example'"`).  
These are not true byte objects but plain strings that were stored with a leading `b'…'`
pattern during export.  

To standardize the dataset, we remove the artificial `b'` and trailing `'` characters from all
string cells where this pattern appears.  
This step preserves the integrity of textual data while ensuring consistency across all
columns that previously displayed byte prefixes.


In [ ]:
# Identify and clean any string values that look like b'example'
# The pattern is purely textual, so we use string replacement
# This preserves numeric fields and normal strings unaffected by the pattern.

def clean_byte_like(value):
    """Remove b'...' or b"..." wrappers from string representations of bytes."""
    if isinstance(value, str):
        if value.startswith("b'") and value.endswith("'"):
            return value[2:-1]
        if value.startswith('b"') and value.endswith('"'):
            return value[2:-1]
    return value

# Apply the cleaning function to all object-type columns
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].apply(clean_byte_like)

print(" Residual byte-like string cleanup completed.")
df.head(3)


✅ Residual byte-like string cleanup completed.


,article_id,created_date,id,identity_attack,insult,obscene,parent_id,parent_text,publication_id,severe_toxicity,sexual_explicit,text,threat,toxicity
0,153145,2016-11-29 17:23:57.762283+00,634903,0.0,0.0,0.0,0,,54,0.0,0.0,"btw, Globe, your new comment section is lame. ...",0.0,0.2
1,379147,2017-09-19 03:02:05.207449+00,5977874,0.0,0.0,0.0,5977787,I get the impression that Boeing is very afra...,54,0.0,0.0,If at first you don't succeed...try again: htt...,0.0,0.0
2,342612,2017-06-10 06:32:21.121964+00,5390534,0.0,0.7,0.0,5388737,Here's a vote for Comey to be prosecuting for ...,55,0.0,0.0,You don't understand what leaking is. By the ...,0.0,0.7


## Parse and Standardize Datetime Column
The `created_date` field contains string-formatted timestamps representing when each comment was posted.
Converting these strings into proper datetime objects allows accurate chronological sorting, filtering,
and time-based analysis. This enables future stages of the project—such as evaluating label drift or
policy consistency over time—to be performed efficiently and correctly.


In [ ]:
# Convert the 'created_date' column from string to datetime
# This parsing automatically interprets timezone offsets and irregular formatting where possible.
# Any unparsable values are coerced to NaT (Not-a-Time) to prevent runtime errors.

df['created_date'] = pd.to_datetime(df['created_date'], errors='coerce')

# Verify conversion success and display a quick summary
print("✅ Datetime conversion completed.")
print("Datatype of 'created_date':", df['created_date'].dtype)
print("Earliest date:", df['created_date'].min())
print("Latest date:", df['created_date'].max())


✅ Datetime conversion completed.
Datatype of 'created_date': datetime64[ns, UTC]
Earliest date: 2015-09-29 10:50:41.987077+00:00
Latest date: 2017-11-11 01:01:10.822969+00:00


## Risk-Focused Sampling Strategy

The goal is to create a balanced, structured sample
of 313 comments from the Civil Comments dataset for re-labeling and evaluation.

We divide the data into three risk tiers based on toxicity scores:

- **High-Risk (≥ 0.50):** 186 comments — clearly toxic  
- **Borderline (0.30 – 0.49):** 62 comments — uncertain or mixed tone  
- **Clean (≤ 0.10):** 62 comments — non-toxic, safe examples  

To make the sampling statistically sound and low-variance, we stratify across *two dimensions*:  
1. The **risk tier** (high-risk, borderline, clean)  
2. The **dominant label** — the label with the highest score among the seven toxicity types  
   (*toxicity, severe_toxicity, obscene, insult, threat, identity_attack, sexual_explicit*)

This combination (e.g., `High-Risk_Insult` or `Borderline_Threat`) creates multi-dimensional strata
and ensures every category is represented in each risk level. It makes the dataset truly
space-filling, balanced, and ready for accurate evaluation work.


In [ ]:
np.random.seed(42)

label_cols = [
    'toxicity', 'severe_toxicity', 'obscene',
    'insult', 'threat', 'identity_attack', 'sexual_explicit'
]

# Targets by risk tier: keep grand total = 313
tier_targets = {'High-Risk': 186, 'Borderline': 62, 'Clean': 62}

# Minimum presence per label (for analysis power):
# We will count a row toward label L if L-score ≥ 0.30 (present).
min_per_label = 15

# Work on a copy to avoid side effects
pool = tiered_df.copy()

# Track selections
selected_idx = set()

# Track how many we have taken from each tier so far
tier_counts = {'High-Risk': 0, 'Borderline': 0, 'Clean': 0}

# Helper: sample rows safely from a candidate subset, honoring tier quotas and avoiding duplicates
def take_from_pool(candidates, needed, tier_name):
    """Return a list of indices selected from 'candidates' up to 'needed',
    without exceeding the remaining tier quota or duplicating rows."""
    remaining_quota = max(0, tier_targets[tier_name] - tier_counts[tier_name])
    k = min(needed, remaining_quota, len(candidates))
    if k <= 0:
        return []
    taken = candidates.sample(n=k, random_state=42).index.tolist()
    return taken

# 1) Ensure ≥ 15 presence per label using High-Risk first (≥0.50), then Borderline (0.30–0.49)
for L in label_cols:
    # Count current presence already selected (initially 0)
    def presence_mask(df):
        return df[L] >= 0.30  # presence definition for later per-label analysis

    current_presence = 0

    # High-Risk pool for this label (presence AND risk_tier == 'High-Risk')
    hr_pool = pool[
        (pool.index.isin(selected_idx) == False) &
        (pool['risk_tier'] == 'High-Risk') &
        (pool[L] >= 0.50)
    ]

    # Take as many as possible toward the min requirement from High-Risk
    need = max(0, min_per_label - current_presence)
    hr_take = take_from_pool(hr_pool, need, 'High-Risk')
    selected_idx.update(hr_take)
    tier_counts['High-Risk'] += len(hr_take)
    current_presence += len(hr_take)

    # If still short, top up from Borderline presence (0.30–0.49) where risk_tier == 'Borderline'
    if current_presence < min_per_label:
        bl_pool = pool[
            (pool.index.isin(selected_idx) == False) &
            (pool['risk_tier'] == 'Borderline') &
            (pool[L] >= 0.30) & (pool[L] < 0.50)
        ]
        need = max(0, min_per_label - current_presence)
        bl_take = take_from_pool(bl_pool, need, 'Borderline')
        selected_idx.update(bl_take)
        tier_counts['Borderline'] += len(bl_take)
        current_presence += len(bl_take)

    # If still short (extremely rare labels), allow spillover from remaining High-Risk presence again,
    # even if not dominant, as long as L >= 0.30 and risk_tier == 'High-Risk' (covers cases with another label slightly higher)
    if current_presence < min_per_label:
        hr_any_pool = pool[
            (pool.index.isin(selected_idx) == False) &
            (pool['risk_tier'] == 'High-Risk') &
            (pool[L] >= 0.30)
        ]
        need = max(0, min_per_label - current_presence)
        hr_any_take = take_from_pool(hr_any_pool, need, 'High-Risk')
        selected_idx.update(hr_any_take)
        tier_counts['High-Risk'] += len(hr_any_take)
        current_presence += len(hr_any_take)

    # NOTE: We do NOT pull presence from Clean for per-label minima,
    # because presence ≥ 0.30 by definition belongs to High-Risk/Borderline thresholds.

# 2) After guaranteeing per-label minima, fill remaining quotas per tier at random (stratified only by tier).
#    This preserves organic composition while completing 186 / 62 / 62.
for tier_name in ['High-Risk', 'Borderline', 'Clean']:
    remaining = max(0, tier_targets[tier_name] - tier_counts[tier_name])
    if remaining == 0:
        continue

    tier_pool = pool[
        (pool.index.isin(selected_idx) == False) &
        (pool['risk_tier'] == tier_name)
    ]

    # If the tier has fewer rows than needed, take all; otherwise sample
    if len(tier_pool) <= remaining:
        take_idx = tier_pool.index.tolist()
    else:
        take_idx = tier_pool.sample(n=remaining, random_state=42).index.tolist()

    selected_idx.update(take_idx)
    tier_counts[tier_name] += len(take_idx)

# 3) Build the final DataFrame, shuffle, and double-check totals
golden_df = pool.loc[list(selected_idx)].sample(frac=1, random_state=42).reset_index(drop=True)

# Safety trims if overshoot (should not happen, but keep consistent with prior pattern)
if len(golden_df) > 313:
    golden_df = golden_df.sample(n=313, random_state=42).reset_index(drop=True)

# 4) Quick verification
print(" Golden Dataset (313 samples) created with per-label presence ≥ 15 and exact tier totals.\n")
print("Tier counts:\n", golden_df['risk_tier'].value_counts().to_string(), "\n")

# Per-label PRESENCE counts (rows where label score ≥ 0.30 anywhere in the row)
presence_summary = {}
for L in label_cols:
    presence_summary[L] = int((golden_df[L] >= 0.30).sum())
print("Per-label presence (≥ 0.30) counts:\n", pd.Series(presence_summary).to_string(), "\n")

# Optional: show dominant_label cross-tab for context (not the enforcement metric)
print("Dominant label distribution (context only):\n")
print(golden_df['dominant_label'].value_counts().to_string())

# 5) Save final dataset
output_path = "/content/drive/MyDrive/Dat490/Dataset/Golden_Subset_v3_min15.csv"
golden_df.to_csv(output_path, index=False)
print(f"\nSaved to {output_path}")


✅ Golden Dataset (313 samples) created with per-label presence ≥ 15 and exact tier totals.

Tier counts:
 risk_tier
High-Risk     186
Borderline     62
Clean          62 

Per-label presence (≥ 0.30) counts:
 toxicity           245
severe_toxicity     18
obscene             48
insult             192
threat              35
identity_attack     53
sexual_explicit     25 

Dominant label distribution (context only):

dominant_label
toxicity           284
insult              11
sexual_explicit      8
obscene              3
identity_attack      3
threat               1

Saved to /content/drive/MyDrive/Dat490/Dataset/Golden_Subset_v3_min15.csv
